# How to improve the safety of driving in MA

## Problem definition: 
Analyze the relationship between the frequency,severity of car accidents and some factors like time period, road sections, weather conditions and locations. Understand the patterns of car crashes that were taking place in MA and what we could do to avoid a car crash. In addition, we are working on joining with another covid dataset to find out the influence of covid on car crashes.

## Used dataset:
* US Accidents (link:https://www.kaggle.com/sobhanmoosavi/us-accidents)
* COVID data(bigquery-public-data:covid19_jhu_csse.summary)

### Overview of carcrash dataset:
* ID: This is a unique identifier of the accident record.
* Start_Time: Shows start time of the accident in local time zone.
* Start_Lat: Shows latitude in GPS coordinate of the start point.
* Start_Lng: Shows longitude in GPS coordinate of the start point.
* County: Shows the county in address field.
* Zipcode: Shows the zipcode in address field.
* Severity: Shows the severity of the accident, a number between 1 and 4, where 1 indicates the least impact on traffic and 4 indicates largest.
* Weather_Condition: Shows the weather condition (rain, snow, thunderstorm, fog, etc.)
* Crossing: A POI annotation which indicates presence of crossing in a nearby location.
* Turning_Loop: A POI annotation which indicates presence of turning_loop in a nearby location.  
(And some similar coloums:Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal)

In [13]:
%%bigquery
SELECT * FROM `ba775-b2-team6.team6.US_Accidents` limit 3

Downloading: 100%|██████████| 3/3 [00:01<00:00,  2.74rows/s]


,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance_mi_,Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1745081,2,2020-11-13 22:50:34+00:00,2020-11-14 00:35:34+00:00,45.505679,-122.780892,45.504015,-122.783203,0.160,A crash has the left lane closed SB at Wilshir...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-1745082,2,2020-10-17 16:33:30+00:00,2020-10-17 20:24:30+00:00,36.771589,-76.257450,36.762995,-76.268282,0.844,Slow traffic on I-64 from Greenbrier Pkwy/Exit...,...,False,False,False,False,False,False,Day,Day,Day,Day
2,A-1745083,3,2017-10-02 15:58:09+00:00,2017-10-02 16:27:56+00:00,34.978237,-82.012154,34.978237,-82.012154,0.000,Accident on I-85 Southbound at Exit 70 I-26.,...,False,False,False,False,False,False,Day,Day,Day,Day


### Overview of COVID dataset:
* Date: Date when the Covid-19 reported.
* Confirmed: 
* Deaths: 
* Recovered: 
* Active: 

In [44]:
%%bigquery
SELECT * FROM `bigquery-public-data.covid19_jhu_csse.summary` LIMIT 3

Downloading: 100%|██████████| 3/3 [00:01<00:00,  2.65rows/s]


,province_state,country_region,date,latitude,longitude,location_geom,confirmed,deaths,recovered,active,fips,admin2,combined_key
0,Mayotte,France,2021-03-15,-12.8275,45.166244,POINT(45.166244 -12.8275),18845,137,2964,15744,None,None,"Mayotte, France"
1,Hamburg,Germany,2021-03-15,53.5511,9.993700,POINT(9.9937 53.5511),55275,1332,49913,4030,None,None,"Hamburg, Germany"
2,Ontario,Canada,2021-06-05,51.2538,-85.323200,POINT(-85.3232 51.2538),542950,8817,525521,8612,None,None,"Ontario, Canada"


### This is an overview of total car crashes between 2016 and 2020 in MA. 
We will focus on the difference of data before 2010 and in 2020 for most of the remainder of the notebook because the covid broke out in early 2020 and the data in 2020 is a special.

In [43]:
%%bigquery
SELECT Distinct EXTRACT(year from start_time)as year,
COUNT(*)AS annual_car_accidents
FROM (
SELECT CAST(Start_Time AS datetime)AS start_time
FROM `ba775-b2-team6.team6.US_Accidents`
WHERE State='MA')
GROUP BY year
ORDER BY year

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.87rows/s]


,year,annual_car_accidents
0,2016,3006
1,2017,5005
2,2018,5192
3,2019,4988
4,2020,6930


## HERE we will do some questions

In [46]:
%%bigquery
SELECT a.Year,a.Monday,b.Tuesday,c.Wednesday,d.Thursday,e.Friday,f.Saturday,g.Sunday
FROM (SELECT EXTRACT(year from start_time)as Year, COUNT(*)AS Monday FROM (SELECT CAST(Start_Time AS datetime)AS start_time FROM `ba775-b2-team6.team6.carcrash_MA` WHERE EXTRACT(year FROM start_time)BETWEEN 2016 AND 2020) WHERE EXTRACT (DAYOFWEEK FROM start_time)=2 GROUP BY Year ORDER BY Year)AS a
LEFT JOIN
(SELECT EXTRACT(year from start_time)as Year, COUNT(*)AS Tuesday FROM (SELECT CAST(Start_Time AS datetime)AS start_time FROM `ba775-b2-team6.team6.carcrash_MA` WHERE EXTRACT(year FROM start_time)BETWEEN 2016 AND 2020) WHERE EXTRACT (DAYOFWEEK FROM start_time)=3 GROUP BY Year ORDER BY Year)AS b USING(Year)
LEFT JOIN
(SELECT EXTRACT(year from start_time)as Year, COUNT(*)AS Wednesday FROM (SELECT CAST(Start_Time AS datetime)AS start_time FROM `ba775-b2-team6.team6.carcrash_MA` WHERE EXTRACT(year FROM start_time)BETWEEN 2016 AND 2020) WHERE EXTRACT (DAYOFWEEK FROM start_time)=4 GROUP BY Year ORDER BY Year)AS c USING(Year)
LEFT JOIN
(SELECT EXTRACT(year from start_time)as Year, COUNT(*)AS Thursday FROM (SELECT CAST(Start_Time AS datetime)AS start_time FROM `ba775-b2-team6.team6.carcrash_MA` WHERE EXTRACT(year FROM start_time)BETWEEN 2016 AND 2020) WHERE EXTRACT (DAYOFWEEK FROM start_time)=5 GROUP BY Year ORDER BY Year)AS d USING(Year)
LEFT JOIN
(SELECT EXTRACT(year from start_time)as Year, COUNT(*)AS Friday FROM (SELECT CAST(Start_Time AS datetime)AS start_time FROM `ba775-b2-team6.team6.carcrash_MA` WHERE EXTRACT(year FROM start_time)BETWEEN 2016 AND 2020) WHERE EXTRACT (DAYOFWEEK FROM start_time)=6 GROUP BY Year ORDER BY Year)AS e USING(Year)
LEFT JOIN
(SELECT EXTRACT(year from start_time)as Year, COUNT(*)AS Saturday FROM (SELECT CAST(Start_Time AS datetime)AS start_time FROM `ba775-b2-team6.team6.carcrash_MA` WHERE EXTRACT(year FROM start_time)BETWEEN 2016 AND 2020) WHERE EXTRACT (DAYOFWEEK FROM start_time)=7 GROUP BY Year ORDER BY Year)AS f USING(Year)
LEFT JOIN
(SELECT EXTRACT(year from start_time)as Year, COUNT(*)AS Sunday FROM (SELECT CAST(Start_Time AS datetime)AS start_time FROM `ba775-b2-team6.team6.carcrash_MA` WHERE EXTRACT(year FROM start_time)BETWEEN 2016 AND 2020) WHERE EXTRACT (DAYOFWEEK FROM start_time)=1 GROUP BY Year ORDER BY Year)AS g USING(Year)
ORDER BY Year

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.36rows/s]


,Year,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,2016,484,637,651,558,538,72,66
1,2017,821,1006,1038,1018,995,52,75
2,2018,886,1045,1082,1014,977,91,97
3,2019,971,1091,1014,947,793,80,92
4,2020,1193,1131,1161,1354,1325,427,339


In [48]:
%%bigquery
SELECT d.m_name AS month ,c.monthly_carcrash_number_2020,c.monthly_carcrash_number_average,(c.monthly_carcrash_number_2020-c.monthly_carcrash_number_average)/c.monthly_carcrash_number_average AS increase_percentage,d.new_confirmed
FROM(
SELECT a.month,a.monthly_carcrash_number_2020,b.monthly_carcrash_number_average
FROM(SELECT EXTRACT(month FROM date)AS month, COUNT(*)AS monthly_carcrash_number_2020 FROM(SELECT EXTRACT(date FROM Start_Time)AS date FROM `ba775-b2-team6.team6.carcrash_MA` WHERE EXTRACT(year FROM Start_Time)=2020) GROUP BY month ORDER BY month)AS a
LEFT JOIN 
(SELECT EXTRACT(month FROM date)AS month, (COUNT(*)/4)AS monthly_carcrash_number_average FROM(SELECT EXTRACT(date FROM Start_Time)AS date FROM `ba775-b2-team6.team6.carcrash_MA` WHERE EXTRACT(year FROM Start_Time)between 2016 and 2019) GROUP BY month ORDER BY month)AS b
ON a.month=b.month ORDER BY month
)AS c
LEFT JOIN 
(SELECT FORMAT_DATE('%B', date) AS m_name,EXTRACT(month FROM date)AS month,sum(new_confirmed)AS new_confirmed FROM `ba775-b2-team6.team6.covid_open_data_MA` WHERE EXTRACT(year from date)=2020 GROUP BY month,m_name ORDER BY month,m_name)AS d
ON c.month=d.month
ORDER BY c.month 

Downloading: 100%|██████████| 12/12 [00:01<00:00, 10.57rows/s]


,month,monthly_carcrash_number_2020,monthly_carcrash_number_average,increase_percentage,new_confirmed
0,January,454,384.00,0.182292,0
1,February,445,360.50,0.234397,0
2,March,643,308.50,1.084279,6620
3,April,999,263.50,2.791271,55585
4,May,842,309.00,1.724919,34760
5,June,843,326.00,1.585890,11917
6,July,340,350.50,-0.029957,8730
7,August,357,423.25,-0.156527,10921
8,September,417,404.75,0.030266,3583
9,October,506,492.00,0.028455,26460
